In [6]:
import numpy as np
import netket as nk

In [22]:
L = 4
graph = nk.graph.Hypercube(length=L, n_dim=2, pbc=False)

# Define the Hilbert space based on this graph
hi = nk.hilbert.Spin(s=0.5, N=graph.n_nodes)

# Define the Sz and SzSz operators
Szs = sum([nk.operator.spin.sigmaz(hi, i) for i in range(hi.size)])/hi.size
Szcorr = sum([nk.operator.spin.sigmaz(hi, i) * nk.operator.spin.sigmaz(hi, j) for (i, j) in graph.edges()])/len(graph.edges())


# Function to create the B-field Hamiltonian
def create_hfield_hamiltonian(hilbert, hfield):
    return sum([hfield * nk.operator.spin.sigmaz(hilbert, i) for i in range(hilbert.size)])

# Define the Heisenberg Hamiltonian
ha_heisenberg = nk.operator.Heisenberg(hilbert=hi, graph=graph)

# Function to compute ground-state energy and expectation values
def compute_expectation_values(hfield):
    # Create the total Hamiltonian
    ha_hfield = create_hfield_hamiltonian(hi, hfield)
    ha = ha_heisenberg + ha_hfield
    
    # Compute the ground-state energy and wavefunction
    evals, evecs = nk.exact.lanczos_ed(ha, k=1, compute_eigenvectors=True)
    exact_gs_energy = evals[0]
    gs = evecs[:, 0]
    
    # Compute the expectation values
    Sz_expectation = gs @ (Szs @ gs)
    SzSz_expectation = gs @ (Szcorr @ gs)
    
    return exact_gs_energy, Sz_expectation, SzSz_expectation

In [25]:
compute_expectation_values(0.6)

(-36.75682826077182, -9.691512892340456e-30, -0.5105115036218305)